https://www.data.go.kr/iim/api/selectAPIAcountView.do


https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057988

In [1]:
import pandas as pd 


#### 요청변수(Request Parameter)

| 항목명(국문) | 항목명(영문)   | 항목크기 | 항목구분 | 샘플데이터               | 항목설명                 |
|---------|-----------|------|------|---------------------|----------------------|
| 고유번호    | pnu       | 19   | 필수   | 1111010100100040007 | 고유번호(8자리 이상)         |
| 기준년도    | stdrYear  | 4    | 옵션   | 2012                | 기준년도(YYYY: 4자리)      |
| 응답결과 형식 | format    | 5    | 옵션   | xml                 | 응답결과 형식(xml 또는 json) |
| 검색건수    | numOfRows | 4    | 옵션   | 10                  | 검색건수                 |
| 페이지 번호  | pageNo    | 4    | 옵션   | 1                   | 페이지 번호               |


#### 출력결과(Response Element)

| 항목명(국문)  | 항목명(영문)        | 항목크기 | 항목구분 | 샘플데이터               | 항목설명     |
|----------|----------------|------|------|---------------------|----------|
| 고유번호     | pnu            | 19   | 필수   | 1111010100100040007 | 고유번호     |
| 법정동코드    | ldCode         | 10   | 옵션   | 1111010100          | 법정동코드    |
| 법정동명     | ldCodeNm       | 300  | 옵션   | 서울특별시 종로구 청운동       | 법정동명     |
| 특수지구분코드  | regstrSeCode   | 1    | 옵션   | 1                   | 특수지구분코드  |
| 특수지구분명   | regstrSeCodeNm | 300  | 옵션   | 일반                  | 특수지구분명   |
| 지번       | mnnmSlno       | 9    | 옵션   | 1-1                 | 지번       |
| 기준년도     | stdrYear       | 4    | 필수   | 2012                | 기준년도     |
| 기준월      | stdrMt         | 2    | 필수   | 09                  | 기준월      |
| 공동주택코드   | aphusCode      | 10   | 옵션   | 1                   | 공동주택코드   |
| 공동주택구분코드 | aphusSeCode    | 1    | 옵션   | 5                   | 공동주택구분코드 |
| 공동주택구분명  | aphusSeCodeNm  | 300  | 옵션   | 다세대                 | 공동주택구분명  |
| 특수지명     | spclLandNm     | 150  | 옵션   | 상암택지개발사업지구 3-7블럭    | 특수지명     |
| 공동주택명    | aphusNm        | 300  | 옵션   | 상암월드컵7단지            | 공동주택명    |
| 전용면적(㎡)  | prvuseAr       | 20   | 옵션   | 84.45               | 전용면적(㎡)  |
| 공시가격(원)  | pblntfPc       | 13   | 옵션   | 480000000           | 공시가격(원)  |
| 데이터기준일자  | lastUpdtDt     | 10   | 옵션   | 2016-09-26          | 데이터기준일자  |


In [2]:
filename = 'ServiceKey.txt'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
        
api_key = get_file_contents(filename)
#print("Our API key is: %s" % (api_key))

In [3]:
import requests

url = 'http://apis.data.go.kr/1611000/nsdi/ApartHousingPriceService/attr/getApartHousingPriceAttr'

In [4]:
#xml
params ={'serviceKey' : api_key,
         'pnu' : '1111010100100040007', 
         'stdrYear' : '2012', 
         'format' : 'xml', 
         'numOfRows' : '10', 
         'pageNo' : '1' }

response_xml = requests.get(url, params=params).text.encode('utf-8')
#response
#print(response.content)

In [5]:
#json
params ={'serviceKey' : api_key,
         'pnu' : '1111010100100040007', 
         'stdrYear' : '2012', 
         'format' : 'json', 
         'numOfRows' : '10', 
         'pageNo' : '1' }

response_json = requests.get(url, params=params).text.encode('utf-8')

In [6]:
import xmltodict
xmltodict.parse(response_xml)

OrderedDict([('response',
              OrderedDict([('numOfRows', '10'),
                           ('pageNo', '1'),
                           ('totalCount', '8'),
                           ('fields',
                            OrderedDict([('field',
                                          [OrderedDict([('pnu',
                                                         '1111010100100040007'),
                                                        ('ldCode',
                                                         '1111010100'),
                                                        ('ldCodeNm',
                                                         '서울특별시 종로구 청운동'),
                                                        ('regstrSeCode', '1'),
                                                        ('regstrSeCodeNm',
                                                         '일반'),
                                                        ('mnnmSlno', '4-7'),
                      

In [7]:
def load_xml_to_dataframe(response, keyword):
    import bs4 as bs
    import urllib.request
    soup = bs.BeautifulSoup(response_xml,'xml')
    
    rows = soup.find_all(keyword)
    columns = rows[0].find_all()
    
    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name) #header
            eachColumn = columns[j].text #value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result

In [8]:
result = load_xml_to_dataframe(response_xml, 'field')

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pnu             8 non-null      object
 1   ldCode          8 non-null      object
 2   ldCodeNm        8 non-null      object
 3   regstrSeCode    8 non-null      object
 4   regstrSeCodeNm  8 non-null      object
 5   mnnmSlno        8 non-null      object
 6   stdrYear        8 non-null      object
 7   stdrMt          8 non-null      object
 8   aphusCode       8 non-null      object
 9   aphusSeCode     8 non-null      object
 10  aphusSeCodeNm   8 non-null      object
 11  aphusNm         8 non-null      object
 12  floorNm         8 non-null      object
 13  hoNm            8 non-null      object
 14  prvuseAr        8 non-null      object
 15  pblntfPc        8 non-null      object
 16  lastUpdtDt      8 non-null      object
dtypes: object(17)
memory usage: 1.2+ KB


In [10]:
result['pblntfPc'] = pd.to_numeric(result['pblntfPc'],errors = 'coerce')

In [11]:
result['pblntfPc'].sum(), result['pblntfPc'].mean()

(4835000000, 604375000.0)

In [12]:
result.shape

(8, 17)

result.columns

cols = pd.read_excel('/Users/reejungkim/Documents/Git/OpenAPI_Ministry_of_land/출력결과(Response Element).xlsx',
             header=1)[['항목명(영문)', '항목명(국문)']]

cols

cols.shape[0]

cols.loc[0]

cols.loc[0][0]

for i in range(0, cols.shape[0]):
    result.rename(columns={cols.loc[i][0]: cols.loc[i][1]}, inplace=True)

In [13]:
result['lng']=126.969069
result['lat']=37.587111

In [14]:

import pydeck as pdk
import streamlit as st


In [15]:
import os
from dotenv import load_dotenv
#.env file to environment
#load_dotenv()
load_dotenv(verbose=True)
token = os.getenv('MAPBOX_API_KEY')


In [16]:
result = result.groupby('ldCode').mean()

In [17]:
#view = pdk.data_utils.compute_view(result[["lng", "lat"]])
#view.pitch = 75
#view.bearing = 60

column_layer = pdk.Layer(
    "ColumnLayer",
    data=result,
    get_position=["lng", "lat"],
    get_elevation=result.pblntfPc.mean(), #"pblntfPc", 
    elevation_scale=10,
    extrude=True,
    radius=100,
   get_fill_color=["pbIntfPc"],
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True,
)


tooltip = {
    "html": "<b>{pblntfPc} 원 </b> ",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

# Set the viewport location
view_state = pdk.ViewState(
    longitude=126.969069,
    latitude=37.587111,
    zoom=15,
    min_zoom=5,
    max_zoom=20,
    pitch=40,
    bearing=-27)


r = pdk.Deck(
    column_layer,
    initial_view_state=view_state,
    tooltip=tooltip,
    map_provider="mapbox",
   # map_style=pdk.map_styles.SATELLITE,
)

r.to_html("column_layer.html")
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27, "latitude": 37.587111, "l…